In [32]:
import re
import sys
from random import random
from math import log
from collections import defaultdict
import numpy as np


tri_counts=defaultdict(int) #counts of all trigrams in input

#We convert set to string for O(1) lookup
validCharacters = set("1234567890abcdefghijklmnopqrstuvwxyz. ")


#Process each line and add start/ending symbols
def preprocess_line(line):
    valid = "##"
    #.lower() automatically changes all capitals to lower.
    for i in line.lower():
        if i in validCharacters:
            if i in set("1234567890"):
                valid += "0"
            else:
                valid += i
    valid+="#"    
    return valid

In [147]:
#get a template dictionary (+1 smoothing)
with open("model-br.en") as f:
    for line in f:
        tri_counts[line[0:3]]=1



In [148]:
#build english model
tri_counts_en=tri_counts.copy()
with open("training.en") as f:
    for line in f:
        line = preprocess_line(line) 
        for j in range(len(line)-(3)):
            trigram = line[j:j+3]
            tri_counts_en[trigram] += 1
sum_en=sum(tri_counts_en.values())
tri_pro_en={k:log(v/sum_en) for k,v in tri_counts_en.items()}

In [149]:
#build Spanish model
tri_counts_es=tri_counts.copy()
with open("training.es") as f:
    for line in f:
        line = preprocess_line(line) 
        for j in range(len(line)-(3)):
            trigram = line[j:j+3]
            tri_counts_es[trigram] += 1
sum_es=sum(tri_counts_es.values())
tri_pro_es={k:log(v/sum_es) for k,v in tri_counts_es.items()}

In [150]:
#build Germany model
tri_counts_de=tri_counts.copy()
with open("training.de") as f:
    for line in f:
        line = preprocess_line(line) 
        for j in range(len(line)-(3)):
            trigram = line[j:j+3]
            tri_counts_de[trigram] += 1
sum_de=sum(tri_counts_de.values())
tri_pro_de={k:log(v/sum_de) for k,v in tri_counts_de.items()}

In [151]:
#get test data
tri_counts_test=defaultdict(int)
with open("test") as f:
    for line in f:
        line = preprocess_line(line) 
        for j in range(len(line)-(3)):
            trigram = line[j:j+3]
            tri_counts_test[trigram] += 1
tri_total=sum(tri_counts_test.values())

In [152]:
#calculate perplexity
entropy_en, entropy_es, entropy_de = 0,0,0
for k,v in tri_counts_test.items():
    entropy_en+=v*tri_pro_en[k]
    entropy_es+=v*tri_pro_es[k]
    entropy_de+=v*tri_pro_de[k]
    
entropy_en/=tri_total
entropy_es/=tri_total
entropy_de/=tri_total

perplexity_en=np.exp(-entropy_en)
perplexity_es=np.exp(-entropy_es)
perplexity_de=np.exp(-entropy_de)

8781.669464298431